In [51]:
import torch

device = torch.device("cuda")

In [52]:
from spuco.utils import set_seed

set_seed(0)

In [53]:
from spuco.robust_train import ERM
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE
root = "data/mnist/"

trainset = SpuCoMNIST(
    root=root,
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.95,
    classes=classes,
    split="train",
    label_noise=0.001
)
trainset.initialize()

testset = SpuCoMNIST(
    root=root,
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()


100%|██████████| 10000/10000 [00:00<00:00, 10209.19it/s]


In [54]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]

valset = SpuCoMNIST(
    root=root,
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="val"
)
valset.initialize()


100%|██████████| 11996/11996 [00:01<00:00, 10240.36it/s]


In [55]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [56]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from spuco.robust_train import ERM
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T
import numpy as np

In [57]:
train_loader = DataLoader(trainset, batch_size=64, shuffle=True)
not_shuffle_train_loader = DataLoader(trainset, batch_size=64, shuffle=False)
test_loader = DataLoader(testset, batch_size=64, shuffle=False)

In [58]:
from spuco.models import model_factory

# model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
model = model_factory("mlp", trainset[0][0].shape, trainset.num_classes).to(device)

from spuco.evaluate import Evaluator

# Initialize the model and optimizer
# optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=False)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
num_epochs = 10
# train_predictions_history = [[] for _ in range(len(trainset))]
# test_predictions_history = [[] for _ in range(len(testset))]
train_acc_history_lst = []
test_acc_history_lst = []

# Training function
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

# Evaluation function
def evaluate(model, data_loader, criterion, device):
    model.eval()
    correct = 0
    total = 0
    predictions = []
    acc_history = []
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
            acc_history.extend((predicted == target).cpu().numpy())
            predictions.extend(predicted.cpu().numpy())
    return correct / total, acc_history

# Calculate forgetability scores
def calculate_forgetability(predictions_history):
    forgetability_scores = []
    for sample_predictions in predictions_history:
        correct_to_incorrect = 0
        for i in range(1, len(sample_predictions)):
            if sample_predictions[i-1] == 1 and sample_predictions[i] == 0:
                correct_to_incorrect += 1

        if sum(sample_predictions) == 0:
            correct_to_incorrect = (num_epochs // 2) + 1
        forgetability_scores.append(correct_to_incorrect)
    return forgetability_scores



for epoch in range(num_epochs):
    train(model, train_loader, optimizer, criterion, device)
    train_acc, train_acc_history = evaluate(model, not_shuffle_train_loader, criterion, device)
    test_acc, test_acc_history = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}")

    train_acc_history_lst.append(train_acc_history)
    test_acc_history_lst.append(test_acc_history)



Epoch 1/10, Train Acc: 0.9508, Test Acc: 0.2309
Epoch 2/10, Train Acc: 0.9524, Test Acc: 0.2683
Epoch 3/10, Train Acc: 0.9533, Test Acc: 0.2742
Epoch 4/10, Train Acc: 0.9566, Test Acc: 0.3309
Epoch 5/10, Train Acc: 0.9574, Test Acc: 0.3469
Epoch 6/10, Train Acc: 0.9564, Test Acc: 0.3407
Epoch 7/10, Train Acc: 0.9587, Test Acc: 0.3647
Epoch 8/10, Train Acc: 0.9591, Test Acc: 0.3613
Epoch 9/10, Train Acc: 0.9613, Test Acc: 0.3964
Epoch 10/10, Train Acc: 0.9622, Test Acc: 0.4178


In [59]:
train_acc_history_lst = np.array(train_acc_history_lst).T
train_forgetability_scores = calculate_forgetability(train_acc_history_lst)


In [60]:
# group accuracy for the last epoch
print("Trainset accuracy for each group")
for group, members in trainset.group_partition.items():
    print(group, end=' ')
    for epoch in range(10):
        group_score = train_acc_history_lst[members, epoch]
        print(f"{group_score.mean():.4f}", end=' ')
    print()

Trainset accuracy for each group
(2, 2) 0.9999 0.9998 0.9989 0.9986 0.9981 0.9931 0.9991 0.9993 0.9985 0.9927 
(0, 0) 0.9990 0.9943 0.9993 0.9985 0.9960 0.9990 0.9997 0.9992 0.9956 0.9990 
(4, 4) 1.0000 1.0000 1.0000 0.9999 0.9997 0.9999 0.9998 0.9964 0.9999 0.9991 
(1, 1) 1.0000 1.0000 0.9998 0.9971 1.0000 1.0000 1.0000 1.0000 0.9999 0.9990 
(3, 3) 1.0000 0.9997 0.9990 0.9988 0.9991 0.9957 0.9919 0.9995 0.9964 0.9998 
(0, 3) 0.0000 0.1127 0.0141 0.2042 0.3028 0.2465 0.4085 0.3662 0.3028 0.2394 
(4, 0) 0.5520 0.6960 0.4240 0.7840 0.3120 0.3680 0.4320 0.2720 0.8720 0.6880 
(3, 1) 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0085 0.1368 
(1, 2) 0.1261 0.0721 0.3243 0.2342 0.4865 0.7838 0.4505 0.3784 0.3604 0.4775 
(0, 2) 0.0000 0.0000 0.0620 0.2481 0.1860 0.0620 0.3023 0.3333 0.3411 0.6279 
(2, 0) 0.0000 0.0000 0.1639 0.0246 0.0246 0.2213 0.2705 0.4426 0.2377 0.3197 
(0, 4) 0.0000 0.0853 0.0310 0.1783 0.3876 0.1163 0.3566 0.8140 0.1628 0.6512 
(1, 3) 0.0000 0.0000 0.0076 0.0

In [61]:
print("Trainset forgetability score each group")
train_forgetability_scores = np.array(train_forgetability_scores)
count = 0
for group, members in trainset.group_partition.items():
    group_score = train_forgetability_scores[members]
    print(group, group_score.mean())


Trainset forgetability score each group
(2, 2) 0.014969009472576306
(0, 0) 0.013621711552459188
(4, 4) 0.004578447794528197
(1, 1) 0.003921141487855354
(3, 3) 0.012324324324324324
(0, 3) 3.9225352112676055
(4, 0) 1.76
(3, 1) 5.17948717948718
(1, 2) 1.882882882882883
(0, 2) 2.4728682170542635
(2, 0) 3.6065573770491803
(0, 4) 2.10077519379845
(1, 3) 4.598484848484849
(3, 2) 2.5955882352941178
(0, 1) 4.2905982905982905
(3, 0) 2.088
(2, 4) 6.0
(4, 1) 5.897435897435898
(4, 3) 3.441860465116279
(3, 4) 4.950819672131147
(1, 0) 2.1338582677165356
(2, 3) 3.496
(2, 1) 4.2727272727272725
(1, 4) 5.583333333333333
(4, 2) 6.0


In [62]:
train_forgetability_scores.shape

(48004,)

In [63]:
train_acc_history_lst[:, -1].shape

(48004,)

In [64]:
# group correct into (0,0), and incorrect into (1,1)

jtt_group_partition = {(0,0): [], (1,1): []}
for i in range(train_acc_history_lst.shape[0]):
    if train_acc_history_lst[i, -1]: # correct
        jtt_group_partition[(0,0)].append(i)
    else: # incorrect
        jtt_group_partition[(1,1)].append(i)

In [76]:
# fgt_group_partition = {(0, i): [] for i in range(7)}
fgt_group_partition = {}
for i in range(train_forgetability_scores.shape[0]):
    # if train_forgetability_scores[i] <= 2: # forgettable
    #     fgt_group_partition[(1,1)].append(i)
    # else: # not forgettable
    #     fgt_group_partition[(0,0)].append(i)

    group = (0, train_forgetability_scores[i].item())
    if group not in fgt_group_partition:
        fgt_group_partition[group] = []
    fgt_group_partition[group].append(i)

In [66]:
import copy

# clone model
model_jtt = model_factory("mlp", trainset[0][0].shape, trainset.num_classes).to(device)
model_fgt = model_factory("mlp", trainset[0][0].shape, trainset.num_classes).to(device)

model_jtt.load_state_dict(copy.deepcopy(model.state_dict()))
model_fgt.load_state_dict(copy.deepcopy(model.state_dict()))


<All keys matched successfully>

In [67]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model_jtt,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:10,  2.25it/s]

Group (0, 0) Accuracy: 99.76359338061465


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:10,  2.22it/s]

Group (0, 1) Accuracy: 15.130023640661939


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:01<00:09,  2.23it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:10<00:00,  2.19it/s]

Group (4, 3) Accuracy: 16.161616161616163


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:11<00:00,  2.20it/s]

Group (4, 4) Accuracy: 100.0


{(0, 0): 99.76359338061465,
 (0, 1): 15.130023640661939,
 (0, 2): 59.810874704491724,
 (0, 3): 16.548463356973997,
 (0, 4): 56.973995271867615,
 (1, 0): 37.89731051344743,
 (1, 1): 100.0,
 (1, 2): 46.3235294117647,
 (1, 3): 9.558823529411764,
 (1, 4): 1.9607843137254901,
 (2, 0): 28.533333333333335,
 (2, 1): 20.8,
 (2, 2): 99.46666666666667,
 (2, 3): 7.466666666666667,
 (2, 4): 0.0,
 (3, 0): 59.79899497487437,
 (3, 1): 17.884130982367758,
 (3, 2): 56.92695214105793,
 (3, 3): 100.0,
 (3, 4): 19.143576826196472,
 (4, 0): 65.74307304785894,
 (4, 1): 3.27455919395466,
 (4, 2): 0.0,
 (4, 3): 16.161616161616163,
 (4, 4): 100.0}

In [68]:
from torch.optim import SGD
from spuco.robust_train import UpSampleERM, DownSampleERM, CustomSampleERM

jtt_train = UpSampleERM(
    model=model_jtt,
    num_epochs=5,
    trainset=trainset,
    batch_size=64,
    group_partition=jtt_group_partition,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=False),
    device=device,
    verbose=True
)
jtt_train.train()

Epoch 0:   0%|          | 0/1444 [00:00<?, ?batch/s]

Epoch 4: 100%|██████████| 1444/1444 [00:07<00:00, 204.99batch/s, accuracy=45.833333333333336%, loss=1.74]


In [69]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model_jtt,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:   0%|          | 0/25 [00:00<?, ?it/s]

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:11,  2.15it/s]

Group (0, 0) Accuracy: 99.76359338061465


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:11,  2.06it/s]

Group (0, 1) Accuracy: 15.130023640661939


...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:11<00:00,  2.05it/s]

Group (4, 3) Accuracy: 16.161616161616163


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:12<00:00,  2.06it/s]

Group (4, 4) Accuracy: 100.0


{(0, 0): 99.76359338061465,
 (0, 1): 15.130023640661939,
 (0, 2): 59.810874704491724,
 (0, 3): 16.548463356973997,
 (0, 4): 56.973995271867615,
 (1, 0): 37.89731051344743,
 (1, 1): 100.0,
 (1, 2): 46.3235294117647,
 (1, 3): 9.558823529411764,
 (1, 4): 1.9607843137254901,
 (2, 0): 28.533333333333335,
 (2, 1): 20.8,
 (2, 2): 99.46666666666667,
 (2, 3): 7.466666666666667,
 (2, 4): 0.0,
 (3, 0): 59.79899497487437,
 (3, 1): 17.884130982367758,
 (3, 2): 56.92695214105793,
 (3, 3): 100.0,
 (3, 4): 19.143576826196472,
 (4, 0): 65.74307304785894,
 (4, 1): 3.27455919395466,
 (4, 2): 0.0,
 (4, 3): 16.161616161616163,
 (4, 4): 100.0}

In [77]:
fgt_group_partition.keys()

for group, members in fgt_group_partition.items():
    print(group, len(members))

(0, 0) 45559
(0, 6) 1354
(0, 1) 713
(0, 2) 300
(0, 3) 78


In [78]:
from torch.optim import SGD
from spuco.robust_train import UpSampleERM, DownSampleERM, CustomSampleERM

fgt_train = UpSampleERM(
    model=model_fgt,
    num_epochs=5,
    trainset=trainset,
    batch_size=64,
    group_partition=fgt_group_partition,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=False),
    device=device,
    verbose=True
)
fgt_train.train()

Epoch 0:   0%|          | 0/3560 [00:00<?, ?batch/s]

Epoch 4: 100%|██████████| 3560/3560 [00:16<00:00, 220.59batch/s, accuracy=63.1578947368421%, loss=0.741]


In [46]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model_fgt,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:07,  3.22it/s]

Group (0, 0) Accuracy: 100.0


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:07,  2.93it/s]

Group (0, 1) Accuracy: 30.260047281323878


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:01<00:07,  2.80it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:08<00:00,  2.93it/s]

Group (4, 3) Accuracy: 6.313131313131313


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:08<00:00,  2.87it/s]

Group (4, 4) Accuracy: 97.97979797979798


{(0, 0): 100.0,
 (0, 1): 30.260047281323878,
 (0, 2): 70.92198581560284,
 (0, 3): 28.13238770685579,
 (0, 4): 72.57683215130024,
 (1, 0): 61.858190709046454,
 (1, 1): 100.0,
 (1, 2): 42.64705882352941,
 (1, 3): 23.03921568627451,
 (1, 4): 7.8431372549019605,
 (2, 0): 38.4,
 (2, 1): 11.733333333333333,
 (2, 2): 99.46666666666667,
 (2, 3): 6.666666666666667,
 (2, 4): 0.0,
 (3, 0): 50.753768844221106,
 (3, 1): 4.534005037783375,
 (3, 2): 55.415617128463474,
 (3, 3): 99.49622166246851,
 (3, 4): 79.34508816120906,
 (4, 0): 36.77581863979849,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 6.313131313131313,
 (4, 4): 97.97979797979798}

In [49]:
evaluator.average_accuracy

95.97156158648662